# Final Project - Group 5

## Group Members
### Xuyang Wang (300389257) 
### Gloria Galadima (300374846)
### Rachel Kim (300394050)
### Liako Kuoe (300381719)

In [ ]:
# Install required libraries
!pip install pandas pyodbc

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyodbc
import os
import re

print(os.getcwd())

C:\Users\xuyan\OneDrive - Douglas College\Douglas\2025_1_Winter\CSIS-3860_Data_Visualization\Final Project


In [ ]:
# Define dataset path
# dataset_path = r"C:\Users\xuyan\OneDrive - Douglas College\Douglas\2025_1_Winter\CSIS-3860_Data_Visualization\Final Project\covid_dataset\covid_19_data_Canada.csv

dataset_path_1 = "covid_dataset/covid_19_data_Canada.csv"
dataset_path_2 = "covid_dataset/source_1/data_class.csv"
dataset_path_3 = "covid_dataset/source_3/Public_COVID-19_Canada.csv"

# Check if the file exists before proceeding
if not os.path.exists(dataset_path_1):
    raise FileNotFoundError(f"Dataset not found at {dataset_path_1}")

# Check if the file exists before proceeding
if not os.path.exists(dataset_path_2):
    raise FileNotFoundError(f"Dataset not found at {dataset_path_2}")

# Check if the file exists before proceeding
if not os.path.exists(dataset_path_3):
    raise FileNotFoundError(f"Dataset not found at {dataset_path_3}")


In [ ]:
# Establish connection to SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;' 
    'DATABASE=CSIS3860-FinalProject-Group5-DataTable;'
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()

cursor

In [ ]:
# Load CSV files
df_1 = pd.read_csv(dataset_path_1, delimiter=",")
df_2 = pd.read_csv(dataset_path_2, delimiter=",")
df_3 = pd.read_csv(dataset_path_3, delimiter=",")

df_1.info()
df_2.info()
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SNo              829 non-null    int64 
 1   ObservationDate  829 non-null    object
 2   Province/State   829 non-null    object
 3   Country/Region   829 non-null    object
 4   Last Update      829 non-null    object
 5   Confirmed        829 non-null    int64 
 6   Deaths           829 non-null    int64 
 7   Recovered        829 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 51.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988 entries, 0 to 1987
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case identifier number  1988 non-null   int64 
 1   Asymptomatic            1988 non-null   object
 2   Age_group               1988 non-null   object
 3   Death                  

In [ ]:
def clean_df(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Remove "Unnamed" columns
    df = df.dropna(axis=1, how='all')                     # Drop all-NaN columns
    df = df.dropna(axis=0, how='all')                     # Drop all-NaN rows
    return df.reset_index(drop=True)

# Apply to each DataFrame
df_1 = clean_df(df_1)
df_2 = clean_df(df_2)
df_3 = clean_df(df_3)

df_1
df_2
df_3


,case_id,provincial_case_id,age,sex,health_region,province,country,date_report,report_week,travel_yn,travel_history_country,locally_acquired
0,1,1,50-59,Male,Toronto,Ontario,Canada,25-01-2020,19-01-2020,1,China,NaN
1,2,2,50-59,Female,Toronto,Ontario,Canada,27-01-2020,26-01-2020,1,China,NaN
2,3,1,40-49,Male,Vancouver Coastal,BC,Canada,28-01-2020,26-01-2020,1,China,NaN
3,4,3,20-29,Female,Middlesex-London,Ontario,Canada,31-01-2020,26-01-2020,1,China,NaN
4,5,2,50-59,Female,Vancouver Coastal,BC,Canada,4/2/2020,2/2/2020,0,NaN,Close Contact
...,...,...,...,...,...,...,...,...,...,...,...,...
3404,3405,655,Not Reported,Not Reported,Interior,BC,Canada,25-03-2020,22-03-2020,Not Reported,NaN,NaN
3405,3406,656,Not Reported,Not Reported,Interior,BC,Canada,25-03-2020,22-03-2020,Not Reported,NaN,NaN
3406,3407,657,Not Reported,Not Reported,Interior,BC,Canada,25-03-2020,22-03-2020,Not Reported,NaN,NaN
3407,3408,658,Not Reported,Not Reported,Interior,BC,Canada,25-03-2020,22-03-2020,Not Reported,NaN,NaN


In [ ]:
# Function to map Pandas data types to SQL Server data types
def map_dtype(dtype):
    dtype = str(dtype)
    if "int" in dtype:
        return "INT"
    elif "float" in dtype:
        return "FLOAT"
    else:
        return "VARCHAR(255)"

# Generate CREATE TABLE query dynamically
table_name_1 = "Covid_19_Data_Canada_After"
table_name_2 = "data_class_After"
table_name_3 = "Public_COVID_19_Canada_After"

# Dictionary to loop through all dataframes and their corresponding table names
dataframes = {
    table_name_1: df_1,
    table_name_2: df_2,
    table_name_3: df_3
}

# Loop to generate and execute CREATE TABLE statements
for table_name, df in dataframes.items():
    columns = ", ".join([f"[{col}] {map_dtype(dtype)}" for col, dtype in zip(df.columns, df.dtypes)])
    create_table_query = f"CREATE TABLE [{table_name}] ({columns});"

    # Execute the CREATE TABLE statement
    try:
        cursor.execute(create_table_query)
        conn.commit()
        print(f"Table '{table_name}' created successfully!")
    except Exception as e:
        print(f"Error creating table '{table_name}': {e}")


Table 'Covid_19_Data_Canada_After' created successfully!
Table 'data_class_After' created successfully!
Table 'Public_COVID_19_Canada_After' created successfully!


In [ ]:
# -------------------------------
# Helper: Convert various date formats to ISO 8601
# -------------------------------
def date_convert_to_iso(val):
    try:
        if "T" in str(val) and len(str(val)) == 19:
            return str(val)
    except:
        pass

    try:
        dt = pd.to_datetime(val, errors="coerce")
        if pd.notnull(dt):
            return dt.strftime("%Y-%m-%dT%H:%M:%S")
    except:
        pass

    return None

# -------------------------------
# Helper: Clean 'Province/State' from df_1
# -------------------------------
province_map_1 = {
    "ON": "Ontario", "QC": "Quebec", "AB": "Alberta", "BC": "British Columbia",
    "MB": "Manitoba", "NB": "New Brunswick", "NL": "Newfoundland and Labrador",
    "NS": "Nova Scotia", "PE": "Prince Edward Island", "SK": "Saskatchewan",
    "NT": "Northwest Territories", "YT": "Yukon", "NU": "Nunavut"
}

def clean_province_1(value):
    if not isinstance(value, str):
        return None

    if any(bad in value.lower() for bad in ["recovered", "princess", "cruise"]):
        return "N/A"

    match = re.search(r",\s*(\w+)$", value)
    if match:
        code = match.group(1)
        return province_map_1.get(code, code)

    return value if value in province_map_1.values() else None

# -------------------------------
# Helper: Clean 'age' column in df_3
# -------------------------------
def clean_age_column(df, column="age"):
    df = df.copy()

    def normalize_age(age):
        age_str = str(age).strip().lower()

        if age_str == "not reported":
            return "Not Reported"

        if age_str in ["<1", "<10", "<18", "<20", "2", "2.0"]:
            return "0-19"

        if "-" in age_str:
            parts = age_str.split("-")
            if all(p.strip().isdigit() for p in parts):
                start, end = int(parts[0]), int(parts[1])
                if end >= 80:
                    return "80+"
                return f"{start}-{end}"

        if age_str.isdigit():
            val = int(age_str)
            if val <= 19: return "0-19"
            elif val <= 29: return "20-29"
            elif val <= 39: return "30-39"
            elif val <= 49: return "40-49"
            elif val <= 59: return "50-59"
            elif val <= 69: return "60-69"
            elif val <= 79: return "70-79"
            else: return "80+"

        return pd.NA

    df[column] = df[column].apply(normalize_age)
    df = df.dropna(subset=[column])
    return df

# -------------------------------
# Helper: Clean region names in df_2
# -------------------------------
region_map = {
    "Atlantic": "East",
    "BC": "West",
    "Ontario": "Central",
    "Prairies": "Central-West"
}

def clean_region_column(df, column="Region"):
    df = df.copy()
    df[column] = df[column].map(lambda x: region_map.get(str(x).strip(), str(x).strip()))
    return df

# -------------------------------
# Helper: Clean province names in df_3
# -------------------------------
province_map_3 = {
    "PEI": "Prince Edward Island", "BC": "British Columbia", "NL": "Newfoundland and Labrador",
    "NS": "Nova Scotia", "NB": "New Brunswick", "QC": "Quebec", "ON": "Ontario",
    "MB": "Manitoba", "SK": "Saskatchewan", "AB": "Alberta", "NT": "Northwest Territories",
    "NU": "Nunavut", "YT": "Yukon", "NWT": "Northwest Territories"
}

def clean_province_column(df, column="province"):
    df = df.copy()
    df[column] = df[column].apply(lambda x: province_map_3.get(str(x).strip(), str(x).strip()))
    return df

# -------------------------------
# Helper: Clean row for SQL INSERT
# -------------------------------
def clean_row(row):
    cleaned = []
    for val in row:
        if pd.isna(val) or val == "":
            cleaned.append(None)
        elif isinstance(val, np.generic):
            cleaned.append(val.item())
        else:
            cleaned.append(val)
    return tuple(cleaned)

# -------------------------------
# Clean df_1
# -------------------------------
df_1["ObservationDate"] = df_1["ObservationDate"].apply(date_convert_to_iso)
df_1["Last Update"] = df_1["Last Update"].apply(date_convert_to_iso)
df_1["Province/State"] = df_1["Province/State"].apply(clean_province_1)
df_1 = df_1[df_1["Province/State"].notnull()]
df_1 = df_1.where(pd.notnull(df_1), None)

# -------------------------------
# Clean df_2
# -------------------------------
df_2 = clean_region_column(df_2, "Region")

# -------------------------------
# Clean df_3
# -------------------------------
df_3 = clean_age_column(df_3, "age")
df_3 = clean_province_column(df_3, "province")
df_3["date_report"] = df_3["date_report"].apply(date_convert_to_iso)
df_3["report_week"] = df_3["report_week"].apply(date_convert_to_iso)


# -------------------------------
# Insert df_1 into SQL
# -------------------------------
for _, row in df_1.iterrows():
    placeholders = ", ".join(["?"] * len(row))
    query = f"INSERT INTO {table_name_1} VALUES ({placeholders})"
    try:
        cursor.execute(query, clean_row(row))
    except Exception as e:
        print(f"Row {idx} failed: {e}")

conn.commit()
print(f"Data inserted into {table_name_1}")

# -------------------------------
# Insert df_2 into SQL
# -------------------------------
for _, row in df_2.iterrows():
    placeholders = ", ".join(["?"] * len(row))
    query = f"INSERT INTO {table_name_2} VALUES ({placeholders})"
    try:
        cursor.execute(query, clean_row(row))
    except Exception as e:
        print(f"Row {idx} failed: {e}")

conn.commit()
print(f"Data inserted into {table_name_2}")

# -------------------------------
# Insert df_3 into SQL
# -------------------------------
for _, row in df_3.iterrows():
    placeholders = ", ".join(["?"] * len(row))
    query = f"INSERT INTO {table_name_3} VALUES ({placeholders})"
    try:
        cursor.execute(query, clean_row(row))
    except Exception as e:
        print(f"Row {idx} failed: {e}")

conn.commit()
print(f"Data inserted into {table_name_3}")


C:\Users\xuyan\AppData\Local\Temp\ipykernel_28480\3487605932.py:12: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(val, errors="coerce")


Data inserted into Covid_19_Data_Canada_After
Data inserted into data_class_After
Data inserted into Public_COVID_19_Canada_After
